In [1]:
import pandas as pd
import sqlite3
import ndjson
from tabulate import tabulate
from Extractor import *

In [2]:
!del chessdb.db

In [3]:
## Create the sqlite3 connection to db
con = sqlite3.connect('chessdb.db')

In [4]:
## Execute SQL script to clean database before loading new data
sql_file = open("chessdb.sql")
sql_as_string = sql_file.read()
cursor = con.cursor()
cursor.executescript(sql_as_string)
sql_file.close()

## Countries

Matching country name to the two letter code.

In [5]:
## Use pandas to upload country table lookup
## file from: https://stefangabos.github.io/world_countries/
countries = pd.read_csv("countries.csv")
countries.to_sql("country", con, if_exists = "replace")

## Users

We pulled users from a large team on lichess.org. We had to do this in order to query for games because you can only query games by looking up games played by username.

In [6]:
## Clean data step
users = pd.read_csv("user_list.csv")
users.drop(['Unnamed: 0'], axis = 1, inplace = True)
## Use pandas to create the new table in the database
users.to_sql("users", con, if_exists = "replace")

## All users that we selected based on parameters on rating and country

We random sample users based on their rating tiers and take out the null/nonexisting countries.

In [7]:
UsersSelected = pd.read_csv("Users_Selected.csv")
UsersSelected.drop(['Unnamed: 0'], axis = 1, inplace = True)
UsersSelected.to_sql("users_selected", con, if_exists = "replace")

## All Games

In [8]:
AllGames = pd.read_csv("all_games.csv")
AllGames.drop(['Unnamed: 0'], axis = 1, inplace = True)
AllGames.to_sql("all_games_raw", con, if_exists = "replace")

## Games with Country included

We clean the country table to exclude any bad countries in their profiles.

In [9]:
all_games_df_with_countries = pd.read_csv("games_with_country.csv")
all_games_df_with_countries.drop(['Unnamed: 0'], axis = 1, inplace = True)
all_games_df_with_countries.to_sql("all_games_country", con, if_exists = "replace")

In [10]:
# Cleaning step
games_cleaned = all_games_df_with_countries[(all_games_df_with_countries['white_country'].str.len() == 2) 
                & (all_games_df_with_countries['black_country'].str.len() == 2)]
games_cleaned.reset_index(inplace = True, drop = True)
games_cleaned.to_sql("all_games_country_selected", con, if_exists = "replace")

## Games with country and wins/10 included

In [11]:
all_games_final = pd.read_csv("All Games Final.csv")
all_games_final.drop(['Unnamed: 0'], axis = 1, inplace = True)
all_games_final.to_sql("all_games_countries_winstreak", con, if_exists = "replace")

In [14]:
con.close()

In [15]:
games_cleaned

,gameid,white_id,white_rating,white_country,white_games,white_win_last_10,white_inaccuracies,white_mistakes,white_blunder,white_acpl,...,black_games,black_win_last_10,black_inaccuracies,black_mistakes,black_blunder,black_acpl,game_type,opening,winner,win_by
0,q95uyLmw,jeryn_fernandes,1155,IN,1253.0,1635530920158,2,0,2,156,...,86.0,-1,0,1,2,72,rapid,Van't Kruijs Opening,black,resign
1,7o4Cl3zU,brindosier,1093,FR,990.0,1632662607717,2,1,11,262,...,1253.0,-1,3,0,10,231,rapid,French Defense: Knight Variation,black,outoftime
2,7ucmFZ4u,alex888rbh75,1104,IN,51.0,1632495202057,1,0,3,95,...,1253.0,-1,3,0,4,131,rapid,Queen's Pawn Game: Franco-Sicilian Defense,white,mate
3,OgveHeqm,bibinator95,1306,HR,629.0,1638791699227,1,1,4,75,...,90.0,-1,1,0,3,55,rapid,Pirc Defense,black,resign
4,fxaqP8Df,heshamelrewesh,732,EG,698.0,1621621307636,2,0,4,163,...,171.0,-1,1,3,4,222,blitz,King's Knight Opening,white,mate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8745,CFrrrzBq,ireonus,2245,ZA,10041.0,1638047176679,2,3,0,32,...,21687.0,-1,7,2,1,55,blitz,Sicilian Defense: Alapin Variation,white,mate
8746,YjQLbjhd,ireonus,2205,ZA,10041.0,1637987527993,3,2,2,55,...,11771.0,-1,3,2,2,30,blitz,French Defense: Exchange Variation,black,resign
8747,OnFaKgro,harsha101,2302,LK,20542.0,1637486152773,9,2,6,37,...,10041.0,-1,12,4,4,50,blitz,Ponziani Opening: Jaenisch Counterattack,white,outoftime
8748,umVzio3K,ireonus,2253,ZA,10041.0,1637479558481,3,4,1,80,...,4072.0,-1,2,2,5,110,blitz,Sicilian Defense: Alapin Variation,white,mate
